In [64]:
import pymysql

conn = pymysql.connect(host = "localhost", port = 3306, db="study", user = "cyhgpl", password = "1596")
try:
    conn
    print("연결성공")
except:
    print("연결실패")



연결성공


In [44]:
import pymysql
with open("./db_config", "r") as f:
    config = f.readlines()


config_dict = {}
for l in config:
    #k, v = l.split('=')
    idx = l.index('=')
    k = l[:idx]
    v = l[idx + 1:]
    config_dict[k] = v.rstrip()

conn = pymysql.connect(**config_dict)  # config_dict 언패킹

try:
    conn
    print("연결성공")
except Exception as e:
    print("연결실패", e)

def pagination(page, page_size=5):
    limit = page_size
    offset = page_size*(page-1)
    with conn.cursor() as cursor:
        #sql = 'insert into `students`(`name`, `age`, `address`) values("지영", "28", "전주");'
        #_sql = 'insert into `students`(`name`, `age`, `address`) values(%s, %s, %s);'
        __sql = 'select * from `students` where `age` > %s limit %s offset %s'
        #cursor.execute(sql)
        #cursor.execute(_sql,("최진","32","천안"))  #데이터를 하나만 넣더라도 튜플이 되게 ("최진",) 이런식으로 넣어준다.
        #cursor.executemany(_sql, [("가영","35","동해"),("민수","40","무안"),("다니엘","35","부산")])
        cursor.execute(__sql,(5, limit, offset))
        data = cursor.fetchall()        # 보통 fetchall 하고 limit과 offset으로 조절한다. 
        #data1 = cursor.fetchone()
        #data2 = cursor.fetchmany(size=5) 
    conn.commit()
    return data

pagination(2)

연결성공


((6, '성춘향', '경기', 27),
 (7, '이몽룡', '대전', 42),
 (9, '지영', '전주', 28),
 (10, '최진', '천안', 20),
 (11, '가영', '동해', 35))

In [45]:
def read_config(p:str) -> dict:
    with open(p, 'r') as f:
        config = f.readlines()
    config_dict = {}
    for l in config:
        #k, v = l.split('=')
        idx = l.index('=')
        k = l[:idx]
        v = l[idx + 1:]
        config_dict[k] = v.rstrip()

    return config_dict


read_config("../db_config copy") # 점 두개는 config


{'host': 'localhost',
 'user': 'cyhgpl',
 'password': '1596',
 'database': 'study',
 'charset': 'utf8mb4'}

In [65]:

with conn.cursor() as cur:
    #sql = 'update students set age=%s where studentid=%s'
    sql = 'delete from students where studentid ="10"'
    cur.execute(sql)
    #cur.executemany(sql, [(55, 2),(10, 5)])
conn.commit()

pagination(2)

((6, '성춘향', '경기', 27),
 (7, '이몽룡', '대전', 42),
 (9, '지영', '전주', 28),
 (11, '가영', '동해', 35),
 (12, '민수', '무안', 40))

In [62]:
conn.close()

In [73]:
with conn.cursor() as cur:
    #sql = 'create table Student(id int not null auto_increment, name varchar(255) not null, email varchar(255) not null, phone varchar(255) not null, major varchar(255) not null, primary key(id));'
    #cur.execute(sql)
    sql_insert = 'insert into Student(name, email, phone, major) values(%s, %s, %s, %s)'
    cur.executemany(sql_insert, [("a", "a@naver.com", "a1234", "math"),("b", "b@naver.com", "b1234", "Computer"),("c", "c@naver.com", "c1234", "Science"),("d", "d@naver.com", "d1234", "English")])
conn.commit()